# Travelling Salesman Problem

The "Travelling Salesman Problem" is well known, and is described in Wikipedia: https://en.m.wikipedia.org/wiki/Travelling_salesman_problem

"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city **exactly once** and returns to the origin city?"

One of the published solutions to it is the "Held-Karp Algorithm": https://en.m.wikipedia.org/wiki/Held%E2%80%93Karp_algorithm

However, in this notebook, we'll attempt to find a solution (at least a "shorter path" if not the "shortest path") from scratch.

At the very least, in what ways can it be solved that aren't just a pure, brute-force search of all possible paths?

## Example Data

There are two data files, one with five cities, one with all 48 statement capital cities in the continental Unites States of America.

Each data file is a matrix of distances between cities, one line per row, all integer values separate by one or more spaces.  The diagonal is all zeroes, since the distance from a city to itself is zero.

One data file has the distances as integers, one has the distances as decimals.

In [1]:
let sample5file = @"sample-data\five_d.txt"
let sample48file = @"sample-data\att48_d.txt"

## Setup

In [2]:
open System.IO

## Data Types

In [3]:
type Row = double list // row of an double matrix
type IndexedRow = int*Row // indexed row with a one-based index (not zero-based)
type Matrix = Row list // double matrix
type GenericMatrix<'a> = 'a list list

// Map of distances between the locations with the two given integer indices.
// Always returns zero if the two input indices are the same index.
// Location indices are one-based, not zero-based.
type DistanceMap = Map<int*int, double>

## Functions

In [4]:
let parseNumberLine (line: string): Row =
    let splitToken = " "
    line.Split(splitToken) |> Array.toList |> List.filter (fun str -> str.Length >= 1) |> List.map System.Double.Parse

let parseNumberLines (lines: string list): Matrix =
    lines |> List.map parseNumberLine |> List.filter (fun row -> row.Length >= 1)

// Used for testing, takes a square of integer values and returns a smaller square.
let takeSquare (size: int) (square: Matrix): Matrix =
    square |> List.take size |> List.map (fun row -> row |> List.take size)

let isEmptyMatrix (matrix: GenericMatrix<'a>): bool =
    matrix |> List.map List.isEmpty |> List.fold (&&) true

In [5]:
let loadDistanceFile (path: string): Matrix =
    let lines = File.ReadLines(path) |> Seq.toList
    if (lines.IsEmpty)
    then [[]]
    else parseNumberLines lines

In [6]:
let mergeMaps(maps: Map<'a,'b> list): Map<'a,'b> =
    maps |> List.map Map.toList |> List.concat |> Map.ofList

In the following, we convert the distances in the matrix to a map from a (row, column) index pair to a distance.

In [7]:
let squareToDistanceMap (square: Matrix): DistanceMap =
    if (isEmptyMatrix square)
    then
        Map.empty
    else
        let indices = Seq.initInfinite (fun n -> n + 1) // 1, 2, 3, 4, ...
        let indexedRows = Seq.zip indices square |> Seq.toList
        let rec rowsToMap (remainingRows: IndexedRow list) (partialMap: DistanceMap): DistanceMap =
            match remainingRows with
            | head::tail ->
                let index, row = head
                let indices = Seq.initInfinite (fun n -> n + 1) // 1, 2, 3, 4, ...
                let indexedDistances = Seq.zip indices row |> Seq.toList
                let newMap: DistanceMap = indexedDistances |> List.map (fun pair -> ((index, fst pair), snd pair)) |> Map.ofList
                rowsToMap tail (mergeMaps [partialMap; newMap])
            | [] -> partialMap
        rowsToMap indexedRows Map.empty

## Path Arithmetic

In [8]:
// A path, with a start city index, intermediate city indices as appropriate, an end city, and the total distance.
type Path = { startCity: int; intermediateCities: int list; endCity: int; distance: double }

let routeToString (route: int list): string =
    route |> List.map (fun n -> n.ToString()) |> String.concat("->")

let pathToString (path: Path): string =
    let pathPart = List.concat [[path.startCity]; path.intermediateCities; [path.endCity]] |> routeToString
    $"[path: {pathPart} | distance: {path.distance}]"

type PathResult = | Success of Path | Failure of string

let isSuccessPathResult (result: PathResult) = match result with | Success _ -> true | Failure _ -> false
let isFailurePathResult (result: PathResult) = match result with | Success _ -> false | Failure _ -> true
let chooseSuccessPath (result: PathResult) = match result with | Success path -> Some(path) | Failure _ -> None

In [9]:
let addPaths (path1: Path) (path2: Path): PathResult =
    if (path1.endCity = path2.startCity)
    then
        Success({startCity=path1.startCity; intermediateCities=List.concat [path1.intermediateCities; [path1.endCity]; path2.intermediateCities]; endCity=path2.endCity; distance=path1.distance + path2.distance})
    else
        Failure(sprintf "path1 does not end where path2 begins: %A vs %A" path1 path2)

In [10]:
let rec addListOfPaths (pathList: Path list): PathResult =
    match pathList with
    | [] -> Failure "no paths to add in list"
    | path::[] -> Success(path)
    | path1::path2::tail ->
        let result = addPaths path1 path2
        match result with
        | Success path -> addListOfPaths (path::tail)
        | Failure _ -> result

In [11]:
let addPathLists (pathList1: Path list) (pathList2: Path list): PathResult list =
    let resultSeq = seq{
        for path1 in pathList1 do
            for path2 in pathList2 do
                addPaths path1 path2
    }
    resultSeq |> Seq.toList

In [12]:
type PathMap = Map<int*int, Path>
type PathListMap = Map<int*int, Path list>

let distanceMapToPathMap (distanceMap: DistanceMap): PathMap =
    distanceMap |> Map.toList |> List.map (fun ((startCity,endCity),distance) -> ((startCity,endCity), {startCity=startCity; intermediateCities=[]; endCity=endCity; distance=distance})) |> Map.ofList

let printPathMap (pathMap: PathMap) =
    for indices, path in (pathMap |> Map.toList) do
        printfn "%A: %A" indices (pathToString path)

let pathMapToPathListMap (pathMap: PathMap): PathListMap =
    pathMap |> Map.toList |> List.map (fun (indices, path) -> (indices, [path])) |> Map.ofList

## Load in the Example Data

Load in the example data from one of the files, and convert it into a double matrix.

In [13]:
let square5 = loadDistanceFile sample5file
square5

[ [ 0, 3, 4, 2, 7 ], [ 3, 0, 4, 6, 3 ], [ 4, 4, 0, 5, 8 ], [ 2, 6, 5, 0, 6 ], [ 7, 3, 8, 6, 0 ] ] HeadOrDefault [ 0, 3, 4, 2, 7 ] HeadOrDefault 0 TailOrNull [ 3, 4, 2, 7 ] HeadOrDefault 3 TailOrNull [ 4, 2, 7 ] HeadOrDefault 4 TailOrNull [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] Head 4 Tail [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] (values) [ 4, 2, 7 ] Head 3 Tail [ 4, 2, 7 ] HeadOrDefault 4 TailOrNull [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] Head 4 Tail [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] (values) [ 4, 2, 7 ] (values) [ 3, 4, 2, 7 ] Head 0 Tail [ 3, 4, 2, 7 ] HeadOrDefault 3 TailOrNull [ 4, 2, 7 ] HeadOrDefault 4 TailOrNull [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] Head 4 Tail [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] (values) [ 4, 2, 7 ] Head 3 Tail [ 4, 2, 7 ] HeadOrDefault 4 TailOrNull [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] Head 4 Tail [ 2, 7 ] HeadOrDefault 2 TailOrNull [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] Head 2 Tail [ 7 ] HeadOrDefault 7 TailOrNull [ ] Head 7 Tail [ ] (values) [ 7 ] (values) [ 2, 7 ] (values) [ 4, 2, 7 ] (values) [ 3, 4, 2, 7 ] (values) [ 0, 3, 4, 2, 7 ] TailOrNull [ [ 3, 0, 4, 6, 3 ], [ 4, 4, 0, 5, 8 ], [ 2, 6, 5, 0, 6 ], [ 7, 3, 8, 6, 0 ] ] HeadOrDefault [ 3, 0, 4, 6, 3 ] HeadOrDefault 3 TailOrNull [ 0, 4, 6, 3 ] HeadOrDefault 0 TailOrNull [ 4, 6, 3 ] HeadOrDefault 4 TailOrNull [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] Head 4 Tail [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] (values) [ 4, 6, 3 ] Head 0 Tail [ 4, 6, 3 ] HeadOrDefault 4 TailOrNull [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] Head 4 Tail [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] (values) [ 4, 6, 3 ] (values) [ 0, 4, 6, 3 ] Head 3 Tail [ 0, 4, 6, 3 ] HeadOrDefault 0 TailOrNull [ 4, 6, 3 ] HeadOrDefault 4 TailOrNull [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] Head 4 Tail [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] (values) [ 4, 6, 3 ] Head 0 Tail [ 4, 6, 3 ] HeadOrDefault 4 TailOrNull [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] Head 4 Tail [ 6, 3 ] HeadOrDefault 6 TailOrNull [ 3 ] Head 6 Tail [ 3 ] (values) [ 6, 3 ] (values) [ 4, 6, 3 ] (values) [ 0, 4, 6, 3 ] (values) [ 3, 0, 4, 6, 3 ] TailOrNull [ [ 4, 4, 0, 5, 8 ], [ 2, 6, 5, 0, 6 ], [ 7, 3, 8, 6, 0 ] ] HeadOrDefault [ 4, 4, 0, 5, 8 ] HeadOrDefault 4 TailOrNull [ 4, 0, 5, 8 ] HeadOrDefault 4 TailOrNull [ 0, 5, 8 ] HeadOrDefault 0 TailOrNull [ 5, 8 ] Head 0 Tail [ 5, 8 ] (values) [ 0, 5, 8 ] Head 4 Tail [ 0, 5, 8 ] HeadOrDefault 0 TailOrNull [ 5, 8 ] Head 0 Tail [ 5, 8 ] (values) [ 0, 5, 8 ] (values) [ 4, 0, 5, 8 ] Head 4 Tail [ 4, 0, 5, 8 ] HeadOrDefault 4 TailOrNull [ 0, 5, 8 ] HeadOrDefault 0 TailOrNull [ 5, 8 

In [14]:
let square1 = takeSquare 1 square5
square1

[ [ 0 ] ] HeadOrDefault [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471824 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method107(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471840 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> ReturnTypeCustomAttributes Microsoft.FSharp.Collections.FSharpList`1[T] Parame

In [15]:
squareToDistanceMap square1

key value (1, 1) Item1 1 Item2 1 0

In [16]:
let square2 = takeSquare 2 square5
square2

[ [ 0, 3 ], [ 3, 0 ] ] HeadOrDefault [ 0, 3 ] HeadOrDefault 0 TailOrNull [ 3 ] HeadOrDefault 3 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663764 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ReturnParameter T IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method107(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663765 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList`1[T] ReturnTypeCustomAttributes Microsoft.FSharp.Collections.FSharpList`1[T] ReturnParameter Microsoft.FSharp.Collections.FSharpList`1[T] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method107(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 3 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Inter

In [17]:
squareToDistanceMap square2

key value (1, 1) Item1 1 Item2 1 0 (1, 2) Item1 1 Item2 2 3 (2, 1) Item1 2 Item2 1 3 (2, 2) Item1 2 Item2 2 0

In [18]:
let square3 = takeSquare 3 square5
square3

[ [ 0, 3, 4 ], [ 3, 0, 4 ], [ 4, 4, 0 ] ] HeadOrDefault [ 0, 3, 4 ] HeadOrDefault 0 TailOrNull [ 3, 4 ] HeadOrDefault 3 TailOrNull [ 4 ] HeadOrDefault 4 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Message The input list was empty. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method107(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Message The input list was empty. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method107(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 4 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Message The input list was empty. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method107(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Message The input list was empty. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method107(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 4 ] Head 3 Tail [ 4 ] HeadOrDefault 4 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method106(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Mes

In [19]:
squareToDistanceMap square3

key value (1, 1) Item1 1 Item2 1 0 (1, 2) Item1 1 Item2 2 3 (1, 3) Item1 1 Item2 3 4 (2, 1) Item1 2 Item2 1 3 (2, 2) Item1 2 Item2 2 0 (2, 3) Item1 2 Item2 3 4 (3, 1) Item1 3 Item2 1 4 (3, 2) Item1 3 Item2 2 4 (3, 3) Item1 3 Item2 3 0

## Calculate the Shortest Path

How do we calculate the shortest path that visits all of the cities in our dataset?  In particular, how do we do it when we don't separately have co-ordinates for the cities that could additionally help by letting us partition the cities into local regions?

Rather than use an existing algorithm, let's try to do something from first principles.

We have $N$ cities $c_{1}$ to $c_{N}$.  We know the distances $d_{m,n}$ between the pairs of cities $c_{m}$ and $c_{n}$.

$d_{m,m}$ is zero for $m$ in $\{1..N\}$, and $d_{m,n} = d_{n,m}$, i.e. this is the symmetric version of the Travelling Salesman Problem.

A solution path is an ordered series of cities $P = \{c_{p_{i}}\}$ for $1 <= i <= N+1$ (to cover all cities, and to finish from where it started), where $c_{p_1} = c_{p_{N+1}}$ and where $\forall j$ such that $1 <= j <= N$, city $c_{j}$ is an element of the series $P$.  We want to find the shortest such path, calculated as the sum of $d_{p_{i},p_{i+1}}$ for $1 <= i < M$.

As the path covers all cities, it can start/finish equivalently at any of the cities, so it's convenient to always start our path at $c_1$.

Suppose we only have one city, $c_{1}$.  The shortest path that visits all cities is simply $c_{1}$ and the path distance is zero.

If we have two cities, $c_{1}$ and $c_{2}$, the shortest path that visits all cities (starting at $c_{1}$) is $c_{1} \rightarrow c_{2} \rightarrow c_{1}$ and the path distance is $d_{1,2} + d_{2,1}$ (= $2 \times d_{1,2}$).

Those are easy, now let's consider three cities, $c_{1}$ to $c_{3}$.  Possible paths are:
* $c_{1} \rightarrow c_{2} \rightarrow c_{3} \rightarrow c_{1}$
* $c_{1} \rightarrow c_{3} \rightarrow c_{2} \rightarrow c_{1}$ (this is just the same as the previous, in reverse order).

For four or more cities, how can we discover shorter/shortest paths?  One heuristic approach, for any pair of cities $c_{m}$ and $c_{n}$, is to find a third city $c_{k}$ such that $d_{m,k}$ + $d_{k,n}$ < $d_{m,n}$.  With that knowledge, we should be able to construct a shorter path than simply travelling from $c_{1}$ to $c_{N}$ to $c_{1}$ again in order.  However, we still have to be careful to avoid visiting the same city twice.  Let's see if we can do that.

In [20]:
let rec shortenPathMap' (N:int) (pathMap: PathMap): PathMap = // try to find shorter paths based visiting an intermediate city - we don't (yet?) check paths involving multiple intermediate cities
    let shortenPath' (N:int) (pathMap: PathMap) (path: Path): Path =
        let alternatePathResultSeq = seq{
            for k in seq{1..N} do
                if ((k <> path.startCity) && (k <> path.endCity))
                then
                    yield addPaths (Map.find (path.startCity,k) pathMap) (Map.find (k,path.endCity) pathMap)
        }
        let alternatePathSeq = alternatePathResultSeq |> Seq.choose chooseSuccessPath
        let minAlternateDistance = alternatePathSeq |> Seq.map (fun path -> path.distance) |> Seq.min
        if (minAlternateDistance < path.distance)
        then
            let minAlternativePath = alternatePathSeq |> Seq.filter (fun path -> path.distance = minAlternateDistance) |> Seq.head // ignoring alternative minimum distance paths for now
            minAlternativePath
        else path 
    let shortenPath = shortenPath' N pathMap
    pathMap |> Map.toList |> List.map (fun (indices, path) -> (indices, shortenPath path)) |> Map.ofList

In [21]:
let rec shortenPathMap (N:int) (pathMap: PathMap): PathMap = // interatively seek the shortest(-ish) path using the heurestic of finding shorter paths between pair of cities
    let shorterPathMap = shortenPathMap' N pathMap
    if (shorterPathMap = pathMap)
    then pathMap
    else shortenPathMap N shorterPathMap

In [22]:
let distN (N:int) (pathMap:PathMap) (pair:int*int): double = (Map.find pair pathMap).distance

let rec routeDistN (N:int) (pathMap:PathMap) (routeList:int list): double =
    match routeList with
    | [] -> 0.0
    | city1::[] -> 0.0
    | city1::city2::tail -> (distN N pathMap (city1,city2)) + (routeDistN N pathMap (city2::tail))

// Expand a route list out to a full route including intermediate cities
let rec expandRouteN (N:int) (pathMap:PathMap) (routeList:int list): int list =
    match routeList with
    | [] -> []
    | city1::[] -> routeList
    | city1::city2::tail ->
        let path12 = Map.find (city1,city2) pathMap
        List.concat [[city1]; path12.intermediateCities; (expandRouteN N pathMap (city2::tail))]

### N = 4

Let's see what we can do, then, for $N=4$.

In [23]:
let N = 4
let square4 = takeSquare N square5
let distanceMap4 = squareToDistanceMap square4
distanceMap4

key value (1, 1) Item1 1 Item2 1 0 (1, 2) Item1 1 Item2 2 3 (1, 3) Item1 1 Item2 3 4 (1, 4) Item1 1 Item2 4 2 (2, 1) Item1 2 Item2 1 3 (2, 2) Item1 2 Item2 2 0 (2, 3) Item1 2 Item2 3 4 (2, 4) Item1 2 Item2 4 6 (3, 1) Item1 3 Item2 1 4 (3, 2) Item1 3 Item2 2 4 (3, 3) Item1 3 Item2 3 0 (3, 4) Item1 3 Item2 4 5 (4, 1) Item1 4 Item2 1 2 (4, 2) Item1 4 Item2 2 6 (4, 3) Item1 4 Item2 3 5 (4, 4) Item1 4 Item2 4 0

In [24]:
let pathMap4 = distanceMapToPathMap distanceMap4

printPathMap pathMap4

(1, 1): "[path: 1->1 | distance: 0]"
(1, 2): "[path: 1->2 | distance: 3]"
(1, 3): "[path: 1->3 | distance: 4]"
(1, 4): "[path: 1->4 | distance: 2]"
(2, 1): "[path: 2->1 | distance: 3]"
(2, 2): "[path: 2->2 | distance: 0]"
(2, 3): "[path: 2->3 | distance: 4]"
(2, 4): "[path: 2->4 | distance: 6]"
(3, 1): "[path: 3->1 | distance: 4]"
(3, 2): "[path: 3->2 | distance: 4]"
(3, 3): "[path: 3->3 | distance: 0]"
(3, 4): "[path: 3->4 | distance: 5]"
(4, 1): "[path: 4->1 | distance: 2]"
(4, 2): "[path: 4->2 | distance: 6]"
(4, 3): "[path: 4->3 | distance: 5]"
(4, 4): "[path: 4->4 | distance: 0]"


In [25]:
let shortenedPathMap4 = shortenPathMap N pathMap4

printPathMap shortenedPathMap4

(1, 1): "[path: 1->1 | distance: 0]"
(1, 2): "[path: 1->2 | distance: 3]"
(1, 3): "[path: 1->3 | distance: 4]"
(1, 4): "[path: 1->4 | distance: 2]"
(2, 1): "[path: 2->1 | distance: 3]"
(2, 2): "[path: 2->2 | distance: 0]"
(2, 3): "[path: 2->3 | distance: 4]"
(2, 4): "[path: 2->1->4 | distance: 5]"
(3, 1): "[path: 3->1 | distance: 4]"
(3, 2): "[path: 3->2 | distance: 4]"
(3, 3): "[path: 3->3 | distance: 0]"
(3, 4): "[path: 3->4 | distance: 5]"
(4, 1): "[path: 4->1 | distance: 2]"
(4, 2): "[path: 4->1->2 | distance: 5]"
(4, 3): "[path: 4->3 | distance: 5]"
(4, 4): "[path: 4->4 | distance: 0]"


In [26]:
shortenedPathMap4 = pathMap4

False

In [27]:
for (indices, path) in (pathMap4 |> Map.toList) do
    let oldDistance = (Map.find indices pathMap4).distance
    let newPath = Map.find indices shortenedPathMap4
    let newDistance = newPath.distance
    if (newDistance <> oldDistance)
    then
        printfn "City segment %A: distance shortened from %f to %f via %A" indices oldDistance newDistance (newPath.intermediateCities)

City segment (2, 4): distance shortened from 6.000000 to 5.000000 via [1]
City segment (4, 2): distance shortened from 6.000000 to 5.000000 via [1]


So, we are **unable** to shorten the path map for $N$ = 4, because the shorter paths are via $c_{1}$, but our path starts and ends at $c_{1}$, so we can't visit it again as an intermediate city in the path.

### Experiments with N=4

Let's play around with the $N$ = 4 case.  We can calculate all permutations (of one visit per city) to find a brute-force minimum.

In [28]:
let dist4 = distN N pathMap4
let routeDist4 = routeDistN N pathMap4
let expandRoute4 = expandRouteN N pathMap4

In [29]:
let allPaths4 = seq{
    for m in seq{2..N} do // 1 is always both the start and end index
        for k in seq{2..N} do
            if (k <> m)
            then
                for n in 2..N do
                    if (n <> m) && (n <> k)
                    then
                        let route = [1; m; k; n; 1]
                        yield (
                            expandRoute4 route,
                            routeDist4 route
                        )
}

for pathAndDist in allPaths4 do
    printfn "[path: %A | distance: %A]" ((fst pathAndDist) |> routeToString) (snd pathAndDist)

[path: "1->2->3->4->1" | distance: 14.0]
[path: "1->2->4->3->1" | distance: 18.0]
[path: "1->3->2->4->1" | distance: 16.0]
[path: "1->3->4->2->1" | distance: 18.0]
[path: "1->4->2->3->1" | distance: 16.0]
[path: "1->4->3->2->1" | distance: 14.0]


In [30]:
let minDistance4 = allPaths4 |> Seq.map (fun pair -> snd pair) |> Seq.min
minDistance4

14

In [31]:
let minPaths4 = allPaths4 |> Seq.filter (fun pair -> (snd pair) = minDistance4)

for pathAndDist in minPaths4 do
    printfn "[path: %A | distance: %A]" ((fst pathAndDist) |> List.map (fun n -> n.ToString()) |> String.concat "->") (snd pathAndDist)

[path: "1->2->3->4->1" | distance: 14.0]
[path: "1->4->3->2->1" | distance: 14.0]


We see that the two shortest paths are just the reverse direction of each other (and oddly in numerical order, which is unusual, but never impossible).

So how could we discover the same shortest paths for ourselves?  Let's capture the shortest path(s) to another city from each city.  To avoid duplication, we'll only do this for city pairs $(c_{m}, c_{n})$ where $m$ < $n$.

In [32]:
let nearestNeighbourPaths4 = seq{
    for m in seq{1..N} do
        let nindices = seq{1..N} |> Seq.filter(fun n -> n <> m)
        let mPaths = seq{
            for n in nindices do
                if m <> n
                then yield (dist4 (m,n))
        }
        let minPath = mPaths |> Seq.min
        let mPairs = Seq.zip nindices mPaths |> Seq.toList
        let minPairs = mPairs |> List.filter (fun pair -> snd pair = minPath)
        let resultSeq = seq {
            for pair in minPairs do
                yield {startCity=m; intermediateCities=[]; endCity=fst pair; distance=snd pair}
        }
        yield (resultSeq |> Seq.toList)
}

for pathSeq in nearestNeighbourPaths4 do
    for path in pathSeq do
        printfn "%A" (pathToString path)

"[path: 1->4 | distance: 2]"
"[path: 2->1 | distance: 3]"
"[path: 3->1 | distance: 4]"
"[path: 3->2 | distance: 4]"
"[path: 4->1 | distance: 2]"


So
* the shortest distance from 1 is to 4 (direct)
* the shortest distance from 2 is to 1 (direct)
* the shortest distance from 3 is to 1 or 2 (both direct)
* the shortest distance from 4 is to 1 (direct).

OK, so we start from $c_{1}$ and go to $c_{4}$.  Now what is the shortest path from $c_{4}$ that doesn't end at $c_{1}$?

In [33]:
for indices in [ (4,3); (4,2) ] do
    printfn "%A" (Map.find indices pathMap4 |> pathToString)

"[path: 4->3 | distance: 5]"
"[path: 4->2 | distance: 6]"


So the shortest distance from $c_{4}$ is to either:
* to $c_{3}$, or
* via $c_{1}$ to $c_{2}$.

There is no difference in overall distance.

What is the shortest path from $c_{2}$ or $c_{3}$ that doesn't end at $c_{1}$ or $c_{4}$?  There is only one such path, from $c_{2}$ to $c_{3}$.

In [34]:
for indices in [ (2,3) ] do
    printfn "%A" (Map.find indices pathMap4 |> pathToString)

"[path: 2->3 | distance: 4]"


Let's compare travelling $c_{2} \rightarrow c_{3} \rightarrow c_{1}$ to travelling $c_{3} \rightarrow c_{2} \rightarrow c_{1}$.

In [35]:
for route in [[2;3;1] ; [3;2;1]] do
    printfn "%A: %A" (route |> routeToString) (routeDist4 route)

"2->3->1": 8.0
"3->2->1": 7.0


Shortest is $c_{3} \rightarrow c_{2} \rightarrow c_{1}$, so altogether our shortest path is:
* $\{c_{1}, c_{4}, c_{3}, c_{2}, c_{1}\}$ or equivalently
* $\{c_{1}, c_{2}, c_{3}, c_{4}, c_{1}\}$

the same as we found with brute force.

### Experiments with N=5

For $N$ = 5, the shortest path is known (see the file 'five_s.txt'), so that gives us a very useful extra point of comparison.  However, we'll ignore this information except as a final check.

In [36]:
let N = 5
let distanceMap5 = squareToDistanceMap square5
distanceMap5

key value (1, 1) Item1 1 Item2 1 0 (1, 2) Item1 1 Item2 2 3 (1, 3) Item1 1 Item2 3 4 (1, 4) Item1 1 Item2 4 2 (1, 5) Item1 1 Item2 5 7 (2, 1) Item1 2 Item2 1 3 (2, 2) Item1 2 Item2 2 0 (2, 3) Item1 2 Item2 3 4 (2, 4) Item1 2 Item2 4 6 (2, 5) Item1 2 Item2 5 3 (3, 1) Item1 3 Item2 1 4 (3, 2) Item1 3 Item2 2 4 (3, 3) Item1 3 Item2 3 0 (3, 4) Item1 3 Item2 4 5 (3, 5) Item1 3 Item2 5 8 (4, 1) Item1 4 Item2 1 2 (4, 2) Item1 4 Item2 2 6 (4, 3) Item1 4 Item2 3 5 (4, 4) Item1 4 Item2 4 0 (4, 5) Item1 4 Item2 5 6 ... (more)

In [37]:
let pathMap5 = distanceMapToPathMap distanceMap5
pathMap5

key value (1, 1) Item1 1 Item2 1 { startCity = 1\n intermediateCities = []\n endCity = 1\n distance = 0.0 } startCity 1 intermediateCities [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method402(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471824 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method402(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method403(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471840 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList

In [38]:
let shortenedPathMap5 = shortenPathMap N pathMap5
shortenedPathMap5

key value (1, 1) Item1 1 Item2 1 { startCity = 1\n intermediateCities = []\n endCity = 1\n distance = 0.0 } startCity 1 intermediateCities [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method402(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471824 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method402(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method403(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471840 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList

In [39]:
shortenedPathMap5 = pathMap5

False

In [40]:
for (indices, path) in (pathMap5 |> Map.toList) do
    let oldDistance = (Map.find indices pathMap5).distance
    let newPath = Map.find indices shortenedPathMap5
    let newDistance = newPath.distance
    if (newDistance <> oldDistance)
    then
        printfn "City segment %A: distance shortened from %f to %f via %A" indices oldDistance newDistance (newPath.intermediateCities)

City segment (1, 5): distance shortened from 7.000000 to 6.000000 via [2]
City segment (2, 4): distance shortened from 6.000000 to 5.000000 via [1]
City segment (3, 5): distance shortened from 8.000000 to 7.000000 via [2]
City segment (4, 2): distance shortened from 6.000000 to 5.000000 via [1]
City segment (5, 1): distance shortened from 7.000000 to 6.000000 via [2]
City segment (5, 3): distance shortened from 8.000000 to 7.000000 via [2]


Some of these shortened paths go through $c_{1}$, so we can't use them.  The rest go via $c_{2}$, so we can use at most one of them in constructing our "shortest path".  We need to be able to revisit our shortened paths, excluding intermediate paths that we can't visit.

In [41]:
let rec reshortenPathMap' (N:int) (pathMap: PathMap) (excludedCities: int list): PathMap = // try to find shorter paths based visiting an intermediate city - we don't (yet?) check paths involving multiple intermediate cities
    let reshortenPath' (N:int) (pathMap: PathMap) (excludedCities: int list) (path: Path): Path =
        let alternatePathResultSeq = seq{
            for k in seq{1..N} do
                if ((k <> path.startCity) && (k <> path.endCity) && (not (excludedCities |> List.contains k)))
                then
                    yield addPaths (Map.find (path.startCity,k) pathMap) (Map.find (k,path.endCity) pathMap)
        }
        let alternatePathSeq = alternatePathResultSeq |> Seq.choose chooseSuccessPath
        let minAlternateDistance = alternatePathSeq |> Seq.map (fun path -> path.distance) |> Seq.min
        if (minAlternateDistance < path.distance)
        then
            let minAlternativePath = alternatePathSeq |> Seq.filter (fun path -> path.distance = minAlternateDistance) |> Seq.head // ignoring alternative minimum distance paths for now
            minAlternativePath
        else path 
    let reshortenPath = reshortenPath' N pathMap excludedCities
    pathMap |> Map.toList |> List.map (fun (indices, path) -> (indices, reshortenPath path)) |> Map.ofList

In [42]:
let rec reshortenPathMap (N:int) (pathMap: PathMap) (excludedCities: int list): PathMap = // interatively seek the shortest(-ish) path using the heurestic of finding shorter paths between pair of cities
    let shorterPathMap = reshortenPathMap' N pathMap excludedCities
    if (shorterPathMap = pathMap)
    then pathMap
    else reshortenPathMap N shorterPathMap excludedCities

In [43]:
let reshortenedPathMap5 = reshortenPathMap N pathMap5 [1]
reshortenedPathMap5

key value (1, 1) Item1 1 Item2 1 { startCity = 1\n intermediateCities = []\n endCity = 1\n distance = 0.0 } startCity 1 intermediateCities [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method402(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663764 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471824 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method402(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method403(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663765 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\tony\.nuget\packages\microsoft.dotnet-interactive\1.0.461101\tools\net8.0\any\FSharp.Core.dll ModuleVersionId 8b3b61bb-207b-2ce2-fc65-52fadafedf72 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=8.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140732134471840 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList

In [44]:
reshortenedPathMap5 = pathMap5

False

In [45]:
for (indices, path) in (pathMap5 |> Map.toList) do
    let oldDistance = (Map.find indices pathMap5).distance
    let newPath = Map.find indices reshortenedPathMap5
    let newDistance = newPath.distance
    if (newDistance <> oldDistance)
    then
        printfn "City segment %A: distance shortened from %f to %f via %A" indices oldDistance newDistance (newPath.intermediateCities)

City segment (1, 5): distance shortened from 7.000000 to 6.000000 via [2]
City segment (3, 5): distance shortened from 8.000000 to 7.000000 via [2]
City segment (5, 1): distance shortened from 7.000000 to 6.000000 via [2]
City segment (5, 3): distance shortened from 8.000000 to 7.000000 via [2]


We can calculate all permutations (of one visit per city) to find a brute-force minimum.

In [46]:
let dist5 = distN N reshortenedPathMap5
let routeDist5 = routeDistN N reshortenedPathMap5
let expandRoute5 = expandRouteN N reshortenedPathMap5

In [47]:
let allPaths5 = seq{
    for m in seq{2..N} do // 1 is always both the start and end index
        for j in seq{2..N} do
            if (j <> m)
            then
                for k in seq{2..N} do
                    if ((k <> m) && (k <> j))
                    then
                        for n in 2..N do
                            if (n <> m) && (n <> j) && (n <> k)
                            then
                                let route = [1; m; j; k; n; 1]
                                yield (
                                    expandRoute5 route,
                                    routeDist5 route
                                )
}

for pathAndDist in allPaths5 do
    printfn "[path: %A | distance: %A]" ((fst pathAndDist) |> routeToString) (snd pathAndDist)

[path: "1->2->3->4->5->2->1" | distance: 24.0]
[path: "1->2->3->2->5->4->1" | distance: 22.0]
[path: "1->2->4->3->2->5->2->1" | distance: 27.0]
[path: "1->2->4->5->2->3->1" | distance: 26.0]
[path: "1->2->5->2->3->4->1" | distance: 20.0]
[path: "1->2->5->4->3->1" | distance: 21.0]
[path: "1->3->2->4->5->2->1" | distance: 26.0]
[path: "1->3->2->5->4->1" | distance: 19.0]
[path: "1->3->4->2->5->2->1" | distance: 24.0]
[path: "1->3->4->5->2->1" | distance: 21.0]
[path: "1->3->2->5->2->4->1" | distance: 22.0]
[path: "1->3->2->5->4->2->1" | distance: 26.0]
[path: "1->4->2->3->2->5->2->1" | distance: 25.0]
[path: "1->4->2->5->2->3->1" | distance: 22.0]
[path: "1->4->3->2->5->2->1" | distance: 20.0]
[path: "1->4->3->2->5->2->1" | distance: 20.0]
[path: "1->4->5->2->3->1" | distance: 19.0]
[path: "1->4->5->2->3->2->1" | distance: 22.0]
[path: "1->2->5->2->3->4->1" | distance: 20.0]
[path: "1->2->5->2->4->3->1" | distance: 24.0]
[path: "1->2->5->2->3->2->4->1" | distance: 25.0]
[path: "1->2->5-

In [48]:
let minDistance5 = allPaths5 |> Seq.map (fun pair -> snd pair) |> Seq.min
minDistance5

19

In [49]:
let minPaths5 = allPaths5 |> Seq.filter (fun pair -> (snd pair) = minDistance5)

for pathAndDist in minPaths5 do
    printfn "[path: %A | distance: %A]" ((fst pathAndDist) |> routeToString) (snd pathAndDist)

[path: "1->3->2->5->4->1" | distance: 19.0]
[path: "1->4->5->2->3->1" | distance: 19.0]


We see again that the two shortest paths are just the reverse direction of each other.  Also, this is the same solution as in the solution file 'file_s.txt'.

Once more, let's capture the shortest path(s) to another city from each city.  To avoid duplication, we'll only do this for city pairs $(c_{m}, c_{n})$ where $m$ < $n$.

In [50]:
let nearestNeighbourPaths5 = seq{
    for m in seq{1..N} do
        let nindices = seq{1..N} |> Seq.filter(fun n -> n <> m)
        let mPaths = seq{
            for n in nindices do
                if m <> n
                then yield (dist5 (m,n))
        }
        let minPath = mPaths |> Seq.min
        let mPairs = Seq.zip nindices mPaths |> Seq.toList
        let minPairs = mPairs |> List.filter (fun pair -> snd pair = minPath)
        let resultSeq = seq {
            for pair in minPairs do
                yield {startCity=m; intermediateCities=[]; endCity=fst pair; distance=snd pair}
        }
        yield (resultSeq |> Seq.toList)
}

for pathSeq in nearestNeighbourPaths5 do
    for path in pathSeq do
        printfn "%A" (pathToString path)

"[path: 1->4 | distance: 2]"
"[path: 2->1 | distance: 3]"
"[path: 2->5 | distance: 3]"
"[path: 3->1 | distance: 4]"
"[path: 3->2 | distance: 4]"
"[path: 4->1 | distance: 2]"
"[path: 5->2 | distance: 3]"


So
* the shortest distance from 1 is to 4 (direct)
* the shortest distance from 2 is to 1 or 5 (both direct)
* the shortest distance from 3 is to 1 or 2 (both direct)
* the shortest distance from 4 is to 1 (direct)
* the shortest distance from 5 is to 2 (direct)

and the paths to $c_{1}$ can only be used as the very final section of the path.

OK, so we start from $c_{1}$ and go to $c_{4}$.  Now what is the shortest path from $c_{4}$ that isn't to $c_{1}$?

In [51]:
for indices in [ (4,2); (4,3); (4,5) ] do
    printfn "%A" (Map.find indices reshortenedPathMap5 |> pathToString)

"[path: 4->2 | distance: 6]"
"[path: 4->3 | distance: 5]"
"[path: 4->5 | distance: 6]"


So the shortest distance from $c_{4}$ is to $c_{3}$.

What the shortest path from $c_{3}$ that isn't to $c_{1}$ or $c_{4}$?

In [52]:
for indices in [ (3,2); (3,5) ] do
    printfn "%A" (Map.find indices reshortenedPathMap5 |> pathToString)

"[path: 3->2 | distance: 4]"
"[path: 3->2->5 | distance: 7]"


It seems that either way we travel from $c_{3}$ to $c_{2}$, so then the shortest path that we find is:
* $c_{1} \rightarrow c_{4} \rightarrow c_{3} \rightarrow c_{2} \rightarrow c_{5} \rightarrow c_{1}$, or
* $c_{1} \rightarrow c_{5} \rightarrow c_{2} \rightarrow c_{3} \rightarrow c_{4} \rightarrow c_{1}$

and the total distance is:

In [53]:
routeDist5 [1;4;3;2;5;1]

20

Oops!  This is not our shortest path of 19 though, which is:
* $c_{1} \rightarrow c_{3} \rightarrow c_{2} \rightarrow c_{5} \rightarrow c_{4} \rightarrow c_{1}$ or
* $c_{1} \rightarrow c_{4} \rightarrow c_{5} \rightarrow c_{2} \rightarrow c_{3} \rightarrow c_{1}$

What went wrong????!!!!  But remember, we are only using a heuristic approach here, there is no guarantee of even reaching a solution, let along the best solution.

We went from $c_{1}$ to $c_{4}$, which is OK, but didn't then go to $c_{5}$ because the path to $c_{3}$ was shorter.  So, we can't just keep taking shortest paths from one step to the next - at least, they might lead to "shorter" path, but not a "shortest" path.

## Alternative Approach for N = 5

Let's consider a different approach for $N=5$.  How about this:
1. Start from $c_{1}$, find all of the paths that are just 1 hop away.
2. Next, find all of the paths that are two hops away.
   Remove any of those paths that 2-hop paths that go to the same place as a 1-hop path, and are no shorter.
3. Keep going in this way, until you have calculated the paths that are up to length "round($N/2$)",
   i.e. up to length 3 for $N=5$.  Now join those remaining paths together, those with a matching end-point and where the total number of hops is 5 and where every city is visited once.

This is still somewhat brute-force, but seeks to break the larger "shortest path" problem into a set of "shortest path" problems that are (approximately) half the length of the full problem, and which hopefully reduces the overall complexity of finding a solution.

In [54]:
let dist5 = distN N pathMap5
let routeDist5 = routeDistN N pathMap5

In [55]:
let shortestPaths_1 =
    seq{
        for n in seq{2..N} do
            yield (Map.find (1,n) pathMap5)
    } |> Seq.toList

for path in shortestPaths_1 do
    printfn "%A" (pathToString path)

"[path: 1->2 | distance: 3]"
"[path: 1->3 | distance: 4]"
"[path: 1->4 | distance: 2]"
"[path: 1->5 | distance: 7]"


In [56]:
let shortestPaths_2' =
    seq {
        for path in shortestPaths_1 do
            for n in seq{2..N} do
                if (n <> path.endCity)
                then
                    yield {startCity=path.startCity; intermediateCities=[path.endCity]; endCity=n; distance=(routeDist5 [path.startCity; path.endCity; n])}
    } |> Seq.toList

// let shortestPaths_2'' = shortestPaths_2' |> List.groupBy (fun path -> path.endCity)

// let filterShortestPaths (paths: Path list) =
//     let minDistance = paths |> List.map (fun path -> path.distance) |> List.min
//     paths |> List.filter (fun path -> path.distance = minDistance)

let shortestPaths_2 = shortestPaths_2' // ' |> List.map (fun group -> snd group |> filterShortestPaths) |> List.concat

for path in shortestPaths_2 do
    printfn "%A" (pathToString path)

"[path: 1->2->3 | distance: 7]"
"[path: 1->2->4 | distance: 9]"
"[path: 1->2->5 | distance: 6]"
"[path: 1->3->2 | distance: 8]"
"[path: 1->3->4 | distance: 9]"
"[path: 1->3->5 | distance: 12]"
"[path: 1->4->2 | distance: 8]"
"[path: 1->4->3 | distance: 7]"
"[path: 1->4->5 | distance: 8]"
"[path: 1->5->2 | distance: 10]"
"[path: 1->5->3 | distance: 15]"
"[path: 1->5->4 | distance: 13]"


In [57]:
let shortestPaths_3' =
    seq {
        for path in shortestPaths_2 do
            for n in seq{2..N} do
                if ((n <> path.endCity) && (not (path.intermediateCities |> List.contains n)))
                then
                    yield {startCity=path.startCity; intermediateCities=List.concat [path.intermediateCities; [path.endCity]]; endCity=n; distance=routeDist5 (List.concat [[path.startCity]; path.intermediateCities; [path.endCity]; [n]])}
    } |> Seq.toList

// let shortestPaths_3'' = shortestPaths_3' |> List.groupBy (fun path -> path.endCity)

let shortestPaths_3 = shortestPaths_3' // ' |> List.map (fun group -> snd group |> filterShortestPaths) |> List.concat

for path in shortestPaths_3 do
    printfn "%A" (pathToString path)

"[path: 1->2->3->4 | distance: 12]"
"[path: 1->2->3->5 | distance: 15]"
"[path: 1->2->4->3 | distance: 14]"
"[path: 1->2->4->5 | distance: 15]"
"[path: 1->2->5->3 | distance: 14]"
"[path: 1->2->5->4 | distance: 12]"
"[path: 1->3->2->4 | distance: 14]"
"[path: 1->3->2->5 | distance: 11]"
"[path: 1->3->4->2 | distance: 15]"
"[path: 1->3->4->5 | distance: 15]"
"[path: 1->3->5->2 | distance: 15]"
"[path: 1->3->5->4 | distance: 18]"
"[path: 1->4->2->3 | distance: 12]"
"[path: 1->4->2->5 | distance: 11]"
"[path: 1->4->3->2 | distance: 11]"
"[path: 1->4->3->5 | distance: 15]"
"[path: 1->4->5->2 | distance: 11]"
"[path: 1->4->5->3 | distance: 16]"
"[path: 1->5->2->3 | distance: 14]"
"[path: 1->5->2->4 | distance: 16]"
"[path: 1->5->3->2 | distance: 19]"
"[path: 1->5->3->4 | distance: 20]"
"[path: 1->5->4->2 | distance: 19]"
"[path: 1->5->4->3 | distance: 18]"


In [58]:
let reversePath (path: Path) =
    {startCity=path.endCity; intermediateCities=List.rev path.intermediateCities; endCity=path.startCity; distance=path.distance}

let fullPaths =
    seq {
        for path1 in shortestPaths_3 do
            for path2 in shortestPaths_2 do // add a '3' to a '2' to get a length '5' path
                if (path1.endCity = path2.endCity)
                then
                    let inter1 = Set.ofList path1.intermediateCities
                    let inter2 = Set.ofList path2.intermediateCities
                    if (Set.intersect inter1 inter2 |> Set.isEmpty) // mustn't pass through the same cities
                    then
                        let path2' = reversePath path2
                        yield {startCity=path1.startCity; intermediateCities=List.concat [path1.intermediateCities; [path1.endCity]; path2'.intermediateCities]; endCity = path2'.endCity; distance=path1.distance + path2'.distance}
    } |> Seq.toList

let shortestPaths = fullPaths |> filterShortestPaths

for path in shortestPaths do
    printfn "%A" (pathToString path)


Error: input.fsx (18,34)-(18,53) typecheck error The value or constructor 'filterShortestPaths' is not defined.

So this gives us our expected shortest path (in both possible directions) of length 19.

## Alternative New Approach

So, one approach would be a brute force approach, test all paths that start at end at $c_{1}$, visit all cities and don't visit any city twice (except $c_{1}$).  So for $N$ cities, that's a path with $N$ segments.  As the start- and end-points are fixed, the number of permutations are $(N-1)!$.  That's how many paths you would have to test for the simplest purely brute force method.

For our 48 city example, the number of permutations is:

In [59]:
let rec fact (n:int): bigint =
    match n with
    | 0 | 1 -> 1I
    | m -> bigint(m) * (fact (m-1))

let permutations48Paths = fact 47
permutations48Paths

258623241511168180642964355153611979969197632389120000000000

In [60]:
double(permutations48Paths)

2.5862324151116818E+59

That's quite a lot!  Can we find a way to reduce the size of the probem?  Suppose we looks at groups of segments of some shorter length, then we piece those together?  Would that help?

At one extreme, we've already done "1 group of 48 segments", and if we did "48 groups of 1 segment", it would really still be the same problem, so we want something in the middle, in a multiplicative sense, like a square root.  $7 \times 7 = 49$, so for 48 we probably want (more conveniently) 6 groups of 8 segments, or 8 groups of 6 segments.

So, let's consider 8 groups of 6 segments.  One of those segments starts at $c_{1}$, with 6 cities to choose, and one ends at $c_{1}$, with 6 cities to choose.  The other 6 groups each have 7 cities to choose (from 47).  So the total number of choices is

In [61]:
let nPr (n:int) (r:int): bigint = (fact n) / (fact (n - r))

let permutations8Groups6Segments = 2I * (nPr 47 6) + 6I * (nPr 47 7)
permutations8Groups6Segments

1917301034880

In [62]:
double(permutations8Groups6Segments)/double(permutations48Paths)

7.413490851313163E-48

You can see that this is a **lot** fewer permutations to cover than for the full brute-force approach, but it's still almost 2 trillion - far too many for a computer that doesn't have many terabytes of memory.  Let's try a smaller number, like 12 - which can be treated as paths of 12 segments that can be broken up into 3 groups each of 4 segments.

In [63]:
let permutations11Paths = fact 11 // {2..12}
permutations11Paths

39916800

In [64]:
let permutations3Groups4Segments = 2I * (nPr 11 4) + (nPr 11 5)
permutations3Groups4Segments

71280

In [65]:
double(permutations3Groups4Segments)/double(permutations11Paths)

0.0017857142857142857

This is much smaller and more tractable, while still not trivial.

In [66]:
let square48 = loadDistanceFile sample48file

In [67]:
let N = 12

let square12 = takeSquare N square48
let distanceMap12 = squareToDistanceMap square12
let pathMap12 = distanceMapToPathMap distanceMap12
let dist12 = distN N pathMap12
let routeDist12 = routeDistN N pathMap12

let intermediateCities = seq{2..N} |> Seq.toList

let intermediateCitiesWithout (exclusions: int list) =
    intermediateCities |> List.filter (fun city -> not (exclusions |> List.contains city))

let segment_1_Permutations =
    seq{
        for c2 in intermediateCitiesWithout [1] do
            for c3 in intermediateCitiesWithout [1;c2] do
                for c4 in intermediateCitiesWithout [1;c2;c3] do
                    for c5 in intermediateCitiesWithout [1;c2;c3;c4] do
                        yield [1;c2;c3;c4;c5]
    } |> Seq.toList

let segment_2_Permutations =
    seq{
        for c1 in intermediateCities do
            for c2 in intermediateCitiesWithout [c1] do
                for c3 in intermediateCitiesWithout [c1;c2] do
                    for c4 in intermediateCitiesWithout [c1;c2;c3] do
                        for c5 in intermediateCitiesWithout [c1;c2;c3;c4] do
                            yield [c1;c2;c3;c4;c5]
    } |> Seq.toList

let segment_3_Permutations =
    seq{
        for c1 in intermediateCities do
            for c2 in intermediateCitiesWithout [c1] do
                for c3 in intermediateCitiesWithout [c1;c2] do
                    for c4 in intermediateCitiesWithout [c1;c2;c3] do
                            yield [c1;c2;c3;c4;1]
    } |> Seq.toList

In [68]:
[segment_1_Permutations.Length; segment_2_Permutations.Length; segment_3_Permutations.Length]

[ 7920, 55440, 7920 ] HeadOrDefault 7920 TailOrNull [ 55440, 7920 ] HeadOrDefault 55440 TailOrNull [ 7920 ] HeadOrDefault 7920 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method402(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663764 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ReturnParameter T IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method402(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\r\n at lambda_method403(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663765 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList`1[T] ReturnTypeCustomAttributes Microsoft.FSharp.Collections.FSharpList`1[T] ReturnParameter Microsoft.FSharp.Collections.FSharpList`1[T] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method403(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 7920 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method402(Closure, FSharpLi

OK, now let's create full paths out of these.

In [69]:
let excludes (aList: 'a list) (exclusions: 'a list): bool =
    let exclusionsSet = exclusions |> Set.ofList
    not (aList |> List.map (fun item -> exclusionsSet |> Set.contains item) |> List.contains true)

In [74]:
let segmentToPath (segment: int list) =
    {startCity = segment[0]; intermediateCities = segment |> List.skip 1 |> List.take (segment.Length - 2); endCity = segment[segment.Length-1]; distance = routeDist12 segment}

In [75]:
let joinPaths (isFullPath: bool) (path1: Path) (path2: Path): Path option =
    if (path1.endCity <> path2.startCity)
    then None
    else
        if (path1.startCity <> 1)
        then None
        else
            if (isFullPath && (path2.endCity <> 1))
            then None
            else
                let allIntermediateCities = path1.intermediateCities @ [path1.endCity] @ path2.intermediateCities
                let allIntermediateCitiesSet = allIntermediateCities |> Set.ofList
                if ((allIntermediateCities.Length = allIntermediateCitiesSet.Count) && (not (allIntermediateCitiesSet |> Set.contains path2.endCity)))
                then Some {startCity = path1.startCity; intermediateCities = allIntermediateCities; endCity = path2.endCity; distance = path1.distance + path2.distance}
                else None

In [76]:
let segment_1_Paths = segment_1_Permutations |> List.map segmentToPath
let segment_2_Paths = segment_2_Permutations |> List.map segmentToPath
let segment_3_Paths = segment_3_Permutations |> List.map segmentToPath

First we calculate the partial paths using only segment 1 and segment 2 paths.

In [77]:
let fullPaths12 =
    seq {
        for path1 in segment_1_Paths do
            for path2 in segment_2_Paths do
                let path12 = joinPaths false path1 path2
                if (path12.IsSome)
                then yield path12.Value
    } |> Seq.toList

In [78]:
fullPaths12.Length

6652800

Let's check that none of the paths in 'fullPaths12' visit the same city twice.

In [79]:
seq {
    for path in fullPaths12 do
        let cities = List.concat [[path.startCity]; path.intermediateCities; [path.endCity]]
        let citiesSet = cities |> Set.ofList
        if (cities.Length <> citiesSet.Count)
        then
            yield path
}

CheckClose,False
LastGenerated,<null>
enum,<null>
pc,0
current,<null>
(values),(empty)


Now we add in segment 3 to get the full paths.

In [80]:
let fullPaths123 =
    seq {
        for path12 in fullPaths12 do
            for path3 in segment_3_Paths do
                let path123 = joinPaths true path12 path3
                if (path123.IsSome)
                then yield path123.Value
    } |> Seq.toList

... and 76min59sec later ...

In [81]:
fullPaths123.Length

39916800

Let's check that none of the paths in 'fullPaths123' visit the same city twice (except the start and end city, of course).

In [84]:
seq {
    for path in fullPaths123 do
        let cities = path.startCity::(path.intermediateCities)
        let citiesSet = cities |> Set.ofList
        if (cities.Length <> citiesSet.Count)
        then
            yield path
}

CheckClose,False
LastGenerated,<null>
enum,<null>
pc,0
current,<null>
(values),(empty)


In [86]:
let minDistance12 = fullPaths123 |> List.map (fun path -> path.distance) |> List.min
minDistance12

19620

In [89]:
let minDistancePaths12 = fullPaths123 |> List.filter (fun path -> path.distance = minDistance12)

for path in minDistancePaths12 do
    printfn "%A" (pathToString path)

"[path: 1->3->2->4->10->5->11->12->6->7->9->8->1 | distance: 19620]"
"[path: 1->8->9->7->6->12->11->5->10->4->2->3->1 | distance: 19620]"


Thus we find two solutions as usual, one being the reverse of the other.